In [1]:
import json
import itertools
import random
import os


training_params = {
    'input_window_size': [60, 120],
#     'scaler': ['min_max_scaler', 'standard_scaler', 'min_max_scaler_1'],
    'scaler': ['none'],
    'normalizer': [True, False],
    'filter_number': [16, 32, 128],
    'conv_window': [(3,)],
    'pooling_window': [(2,)],
    'dropout_rate': [[0.0, 0.0, 0.0]],
    'activation': ['elu'],
    'dense_activation': ['softmax'],
    'optimizer': [('rmsprop', 1)],
    'loss': ['categorical_crossentropy'],
    'layers': [0, 1],
    'l1_value': [0],
    'l2_value': [0],
    'epochs': [100],
    'batch_size': [3500]
}

def get_random_set(training_params):
    return {
        'input_window_size': random.choice(training_params['input_window_size']),
        'scaler': random.choice(training_params['scaler']),
        'normalizer': random.choice(training_params['normalizer']),
        'filter_number': random.choice(training_params['filter_number']),
        'conv_window': random.choice(training_params['conv_window']),
        'pooling_window': random.choice(training_params['pooling_window']),
        'dropout_rate': random.choice(training_params['dropout_rate']),
        'activation': random.choice(training_params['activation']),
        'dense_activation': random.choice(training_params['dense_activation']),
        'optimizer': random.choice(training_params['optimizer']),
        'loss': random.choice(training_params['loss']),
        'layers': random.choice(training_params['layers']),
        'l1_value': random.choice(training_params['l1_value']),
        'l2_value': random.choice(training_params['l2_value']),
        'epochs': training_params['epochs'],
        'batch_size': random.choice(training_params['batch_size'])
      }

def generate_experiments(factors, repetitions):
    # Get list of list of factors
    list_of_factor_values = [factors[factor_name] for factor_name in factors.keys()]
    # Get as permutation and multiply by number of repetitions
    experiments_permutations = list(itertools.product(*list_of_factor_values)) * repetitions
    # Randomize the list
    return random.sample(experiments_permutations, len(experiments_permutations))

class MultiDimensionalArrayEncoder(json.JSONEncoder):
    def encode(self, obj):
        def hint_tuples(item):
            if isinstance(item, tuple):
                return {'__tuple__': True, 'items': item}
            if isinstance(item, list):
                return [hint_tuples(e) for e in item]
            if isinstance(item, dict):
                return {key: hint_tuples(value) for key, value in item.items()}
            else:
                return item

        return super(MultiDimensionalArrayEncoder, self).encode(hint_tuples(obj))

def hinted_tuple_hook(obj):
    if '__tuple__' in obj:
        return tuple(obj['items'])
    else:
        return obj


with open('/home/irene/Documents/Datos-Tesis/experiments_without_scaler.json', 'w') as file_obj:
    enc = MultiDimensionalArrayEncoder()
    experiments_json = enc.encode(generate_experiments(training_params, 3))
    file_obj.write(experiments_json)